In [16]:
import numpy as np
total = [0]
def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
            total[0] += len(row)*2
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.random.rand(3, 2)
b = np.random.rand(2, 3)

print(np.array_equal(a@b, my_matmult(a,b)))


2
2
2
2
2
2
2
2
2
True


In [17]:
def tensor_product(m1, m2):
  ans = np.empty( (0, m1.shape[-2], m2.shape[-1]) )
  for i in range(m1.shape[0]):
    matmult = np.expand_dims(my_matmult(m1[i], m2[i]), axis=0)
    ans = np.vstack((ans, matmult))
  return ans

a = np.random.rand(2, 3, 4)
b = np.random.rand(2, 4, 2)

print(np.allclose(tensor_product(a,b), a @ b))


4
4
4
4
4
4
4
4
4
4
4
4
True


In [18]:

def nd_tensor_product(m1, m2):
  ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
  ans = np.empty(ans_shape)
  for i in range(m1.shape[0]):
    if len(m1.shape) == 3:
        ans[i] = my_matmult(m1[i], m2[i])

    else:
        ans[i] = nd_tensor_product(m1[i], m2[i])
  return ans

a = np.random.rand(12, 4, 64)
b = np.random.rand(12, 64, 4)

total = [0]

print(np.allclose(nd_tensor_product(a,b), a @ b))
print(total)

64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
True
[24576]


In [23]:
def tensor_elements(tensor_shape):
    ans = 1
    for dimention in tensor_shape:
        ans *= dimention
    return ans


# max(out, key=len)

opp_time_func = {
    'add': lambda i, o: tensor_elements(o),
    'subtract': lambda i, o: tensor_elements(o),
    'multiply': lambda i, o: tensor_elements(o),
    'divide': lambda i, o:  tensor_elements(o),
    'rsqrt': lambda i, o: tensor_elements(o),
    'power': lambda i, o: tensor_elements(o),
    'mean': lambda i, o: 0, # TODO
    'nop': lambda i, o: 0, #reshape(Non-Computational)
    'less': lambda i, o: 1,
    'where': lambda i, o: 1,
    'take': lambda i, o: 1,
    'softmax': lambda i, o: 6*i[0][-1]*i[0][-2],
    # 'matmul': lambda i, o: tensor_elements(i[0])*i[0][-1]*i[1][-2],
    'matmul': lambda i, o: tensor_elements(i[0])*i[1][-2]*2,
    'transpose': lambda i, o: 0,
    'split': lambda i, o: 0,
    'dense': lambda i, o: 0,
    'tanh': lambda i, o: 0,
}

input_shapes = [[12, 4, 64], [12, 4, 64]]
output_shape = []
cycles = opp_time_func['matmul'](input_shapes, output_shape)
cycles

3072

In [22]:
12*4*4*64*2

24576

In [24]:
12*4*64

3072